Generate comparison from different models

In [13]:
import os
import cv2
import yaml
import numpy as np
from ultralytics import YOLO
from huggingface_hub import hf_hub_download

# ==========================================================
# 1) LIST OF MODELS (UP TO 5 MODELS)
# ==========================================================
models = [
    {
        "name": "YOLOv8n-S3",
        "repo": "rubbi194/trashdetection",
        "file": "scenario3/yolov8n/best.pt"
    },
    {
        "name": "YOLOv8s-S3",
        "repo": "rubbi194/trashdetection",
        "file": "scenario3/yolov8s/best.pt"
    },
    {
        "name": "YOLOv8m-S3",
        "repo": "rubbi194/trashdetection",
        "file": "scenario3/yolov8m/best.pt"
    },
    {
        "name": "YOLOv8l-S3",
        "repo": "rubbi194/trashdetection",
        "file": "scenario3/yolov8l/best.pt"
    },
    {
        "name": "YOLOv8x-S3",
        "repo": "rubbi194/trashdetection",
        "file": "scenario3/yolov8x/best.pt"
    },
]

# ==========================================================
# 2) Your fixed test images & labels path
# ==========================================================

test_images_dir = r"C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test\images"
labels_dir = r"C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test\labels"

os.makedirs("comparison_multi", exist_ok=True)

# ==========================================================
# Helper – draw YOLO boxes with internal label banner
# ==========================================================
def draw_boxes(img, boxes, color, label_name):
    H, W = img.shape[:2]

    # ========= Draw top banner =========
    banner_h = 50
    overlay = img.copy()
    cv2.rectangle(overlay, (0, 0), (W, banner_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, 0.4, img, 0.6, 0)

    cv2.putText(
        img, label_name, (10, 35),
        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3, cv2.LINE_AA
    )

    # ========= Draw bounding boxes =========
    for b in boxes:
        cls, xc, yc, w, h = b

        x1 = int((xc - w/2) * W)
        y1 = int((yc - h/2) * H)
        x2 = int((xc + w/2) * W)
        y2 = int((yc + h/2) * H)

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(
            img, str(cls), (x1, max(0, y1 - 5)),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2
        )

    return img

# ==========================================================
# Load all models
# ==========================================================
loaded_models = []

for m in models:
    print("Loading:", m["name"])
    model_path = hf_hub_download(
        repo_id=m["repo"],
        filename=m["file"],
        repo_type="model"
    )
    loaded_models.append((m["name"], YOLO(model_path)))

# ==========================================================
# 3) Process entire test set
# ==========================================================
image_exts = [".jpg", ".png", ".jpeg"]

for fname in os.listdir(test_images_dir):
    if not any(fname.lower().endswith(ext) for ext in image_exts):
        continue

    name = os.path.splitext(fname)[0]
    img_path = os.path.join(test_images_dir, fname)

    img = cv2.imread(img_path)
    if img is None:
        print("Cannot load:", img_path)
        continue

    # ------------------------------------------------------
    # Ground truth
    # ------------------------------------------------------
    label_file = os.path.join(labels_dir, name + ".txt")
    gt_boxes_raw = []

    if os.path.exists(label_file):
        with open(label_file, "r") as f:
            for line in f.readlines():
                gt_boxes_raw.append(list(map(float, line.split())))

    gt_boxes = [(b[0], b[1], b[2], b[3], b[4]) for b in gt_boxes_raw]

    gt_img = img.copy()
    gt_img = draw_boxes(gt_img, gt_boxes, (0, 255, 0), "GROUND TRUTH")

    panels = [gt_img]

    # ------------------------------------------------------
    # Predictions for each model
    # ------------------------------------------------------
    for (model_name, model) in loaded_models:
        pred = model(img_path)[0]

        pred_boxes = []
        for r in pred.boxes:
            cls = int(r.cls)
            xc, yc, w, h = r.xywhn[0]
            pred_boxes.append((cls, xc, yc, w, h))

        model_img = img.copy()
        model_img = draw_boxes(
            model_img,
            pred_boxes,
            (0, 0, 255),
            f"PREDICTION – {model_name}"
        )

        panels.append(model_img)

    # ------------------------------------------------------
    # Combine horizontally
    # ------------------------------------------------------
    heights = [p.shape[0] for p in panels]
    max_h = max(heights)

    resized_panels = [
        cv2.resize(p, (p.shape[1], max_h)) for p in panels
    ]

    combined = np.hstack(resized_panels)

    out_path = f"comparison_multi/{name}_multi_compare.jpg"
    cv2.imwrite(out_path, combined)
    print("Saved:", out_path)

print("\n✔ DONE — saved to comparison_multi/")


Loading: YOLOv8n-S3
Loading: YOLOv8s-S3
Loading: YOLOv8m-S3
Loading: YOLOv8l-S3
Loading: YOLOv8x-S3

image 1/1 C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test\images\000101_jpg.rf.fbf75c64095342160f45a3f55d72e725.jpg: 320x320 4 trashs, 11.0ms
Speed: 3.0ms preprocess, 11.0ms inference, 5.0ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test\images\000101_jpg.rf.fbf75c64095342160f45a3f55d72e725.jpg: 320x320 5 trashs, 19.0ms
Speed: 1.0ms preprocess, 19.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test\images\000101_jpg.rf.fbf75c64095342160f45a3f55d72e725.jpg: 320x320 6 trashs, 20.0ms
Speed: 1.0ms preprocess, 20.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test\images\000101_jpg.rf.fbf75c64095342160f45a3f55d72e725

In [3]:
import cv2
import os
import numpy as np

folder = r"C:\Users\User\Documents\Rabab\dataset\Scenario 1\My-First-Project-6"

resolutions = []
blur_scores = []
brightness_scores = []

# Allowed image extensions
valid_ext = ('.jpg', '.jpeg', '.png', '.bmp')

for root, dirs, files in os.walk(folder):
    for img_name in files:
        if img_name.lower().endswith(valid_ext):
            path = os.path.join(root, img_name)
            
            img = cv2.imread(path)
            if img is None:
                continue
            
            # Resolution
            h, w, _ = img.shape
            resolutions.append((w, h))

            # Blur
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            blur_score = cv2.Laplacian(gray, cv2.CV_64F).var()
            blur_scores.append(blur_score)

            # Brightness
            brightness = np.mean(gray)
            brightness_scores.append(brightness)

print("===== RESULTS =====")
print("Total Images:", len(resolutions))

if len(resolutions) > 0:
    print("Min Resolution:", min(resolutions))
    print("Max Resolution:", max(resolutions))
    print(
        "Average Resolution (Width x Height):",
        np.mean([r[0] for r in resolutions]),
        "x",
        np.mean([r[1] for r in resolutions])
    )
else:
    print("No images found.")

# Thresholds (you can keep these as default)
blur_threshold = 100         # <100 = blurry
brightness_threshold = 80    # <80 = low-light

print("Blurry Images:", sum(b < blur_threshold for b in blur_scores))
print("Low-Light Images:", sum(b < brightness_threshold for b in brightness_scores))


===== RESULTS =====
Total Images: 10028
Min Resolution: (640, 640)
Max Resolution: (640, 640)
Average Resolution (Width x Height): 640.0 x 640.0
Blurry Images: 4
Low-Light Images: 0


In [4]:
import cv2
import os
import numpy as np

folder = r"C:\Users\User\Documents\Rabab\dataset\Scenario 2\My-First-Project-8"

resolutions = []
blur_scores = []
brightness_scores = []

# Allowed image extensions
valid_ext = ('.jpg', '.jpeg', '.png', '.bmp')

for root, dirs, files in os.walk(folder):
    for img_name in files:
        if img_name.lower().endswith(valid_ext):
            path = os.path.join(root, img_name)
            
            img = cv2.imread(path)
            if img is None:
                continue
            
            # Resolution
            h, w, _ = img.shape
            resolutions.append((w, h))

            # Blur
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            blur_score = cv2.Laplacian(gray, cv2.CV_64F).var()
            blur_scores.append(blur_score)

            # Brightness
            brightness = np.mean(gray)
            brightness_scores.append(brightness)

print("===== SCENARIO 2 RESULTS =====")
print("Total Images:", len(resolutions))

if len(resolutions) > 0:
    print("Min Resolution:", min(resolutions))
    print("Max Resolution:", max(resolutions))
    print(
        "Average Resolution (Width x Height):",
        np.mean([r[0] for r in resolutions]),
        "x",
        np.mean([r[1] for r in resolutions])
    )
else:
    print("No images found.")

# Thresholds (default)
blur_threshold = 100
brightness_threshold = 80

print("Blurry Images:", sum(b < blur_threshold for b in blur_scores))
print("Low-Light Images:", sum(b < brightness_threshold for b in brightness_scores))


===== SCENARIO 2 RESULTS =====
Total Images: 25433
Min Resolution: (640, 640)
Max Resolution: (640, 640)
Average Resolution (Width x Height): 640.0 x 640.0
Blurry Images: 397
Low-Light Images: 937


In [7]:
import cv2
import os
import numpy as np

folder = r"C:\Users\User\Documents\Rabab\dataset\Scenario 3\My-First-Project-11"

resolutions = []
blur_scores = []
brightness_scores = []

# Allowed image extensions
valid_ext = ('.jpg', '.jpeg', '.png', '.bmp')

for root, dirs, files in os.walk(folder):
    for img_name in files:
        if img_name.lower().endswith(valid_ext):
            path = os.path.join(root, img_name)
            
            img = cv2.imread(path)
            if img is None:
                continue
            
            # Resolution
            h, w, _ = img.shape
            resolutions.append((w, h))

            # Blur detection
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            blur_score = cv2.Laplacian(gray, cv2.CV_64F).var()
            blur_scores.append(blur_score)

            # Brightness detection
            brightness = np.mean(gray)
            brightness_scores.append(brightness)

print("===== SCENARIO 3 RESULTS =====")
print("Total Images:", len(resolutions))

if len(resolutions) > 0:
    print("Min Resolution:", min(resolutions))
    print("Max Resolution:", max(resolutions))
    print(
        "Average Resolution (Width x Height):",
        np.mean([r[0] for r in resolutions]),
        "x",
        np.mean([r[1] for r in resolutions])
    )
else:
    print("No images found.")

# Thresholds
blur_threshold = 100         # <100 = blurry
brightness_threshold = 80    # <80 = low-light

print("Blurry Images:", sum(b < blur_threshold for b in blur_scores))
print("Low-Light Images:", sum(b < brightness_threshold for b in brightness_scores))


===== SCENARIO 3 RESULTS =====
Total Images: 25433
Min Resolution: (572, 572)
Max Resolution: (2048, 1365)
Average Resolution (Width x Height): 1773.472968190933 x 1041.514764282625
Blurry Images: 19359
Low-Light Images: 2424


In [12]:
import os
import glob
import shutil
import numpy as np
from ultralytics import YOLO

# ==========================
# 1. CONFIGURATION
# ==========================

# Folder with test images (Scenario 1, your test set)
IMG_DIR = r"C:\Users\User\Documents\Rabab\dataset\Scenario 1\My-First-Project-6\test\images"
LABEL_DIR = r"C:\Users\User\Documents\Rabab\dataset\Scenario 1\My-First-Project-6\test\labels"

# Output folders (will be created if not exist)
OUT_ROOT = r"C:\Users\User\Documents\Rabab\dataset\Scenario 1\analysis_detection"
SUCCESS_DIR = os.path.join(OUT_ROOT, "success_detection")
PARTIAL_DIR = os.path.join(OUT_ROOT, "partial_detection")
MISSED_DIR = os.path.join(OUT_ROOT, "missed_detection")

os.makedirs(SUCCESS_DIR, exist_ok=True)
os.makedirs(PARTIAL_DIR, exist_ok=True)
os.makedirs(MISSED_DIR, exist_ok=True)

# Base directory where your YOLO runs & models live
BASE_MODEL_DIR = r"C:\Users\User\Documents\Rabab"

# IoU thresholds
IOU_SUCCESS = 0.5   # good detection
IOU_PARTIAL = 0.1   # weak detection, but some overlap


# ==========================
# 2. Find best.pt automatically
# ==========================

def find_best_weight(base_dir, keywords=("scenario1", "yolov8l")):
    """
    Search recursively for 'best.pt' under base_dir.
    Prefer paths that contain all given keywords.
    """
    candidates = []
    for root, dirs, files in os.walk(base_dir):
        for f in files:
            if f.lower() == "best.pt":
                full_path = os.path.join(root, f)
                candidates.append(full_path)

    if not candidates:
        raise FileNotFoundError(f"No best.pt found under {base_dir}")

    # Try to pick the one that matches all keywords
    keywords = [k.lower() for k in keywords]
    for path in candidates:
        low = path.lower()
        if all(k in low for k in keywords):
            print(f"[INFO] Using weight (matched keywords): {path}")
            return path

    # Fallback: just use the first best.pt found
    print("[WARN] No best.pt matched all keywords; using first found:")
    print("       ", candidates[0])
    return candidates[0]


MODEL_PATH = find_best_weight(BASE_MODEL_DIR, keywords=("scenario1", "yolov8l"))
model = YOLO(MODEL_PATH)


# ==========================
# 3. Helper functions
# ==========================

def xywhn_to_xyxy(box, img_w, img_h):
    """
    Convert YOLO normalized [cx, cy, w, h] to pixel [x1, y1, x2, y2].
    """
    cx, cy, w, h = box
    x1 = (cx - w / 2) * img_w
    y1 = (cy - w / 2) * img_h  # fix: y uses h
    y1 = (cy - h / 2) * img_h
    x2 = (cx + w / 2) * img_w
    y2 = (cy + h / 2) * img_h
    return [x1, y1, x2, y2]


def iou(boxA, boxB):
    """
    Compute Intersection over Union (IoU) between two boxes.
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH

    if interArea == 0:
        return 0.0

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    return interArea / (boxAArea + boxBArea - interArea)


# ==========================
# 4. Main loop over images
# ==========================

image_paths = (
    glob.glob(os.path.join(IMG_DIR, "*.jpg")) +
    glob.glob(os.path.join(IMG_DIR, "*.png")) +
    glob.glob(os.path.join(IMG_DIR, "*.jpeg"))
)

print(f"Total test images found: {len(image_paths)}")

for img_path in image_paths:
    img_name = os.path.basename(img_path)
    name_wo_ext = os.path.splitext(img_name)[0]

    # Ground-truth label path
    label_path = os.path.join(LABEL_DIR, name_wo_ext + ".txt")

    if not os.path.exists(label_path):
        print(f"[WARN] No GT label for {img_name}, skipping.")
        continue

    # ---- 4.1 Read ground-truth YOLO labels ----
    gt_boxes = []
    gt_classes = []

    with open(label_path, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls = int(parts[0])
            cx, cy, w, h = map(float, parts[1:])
            gt_classes.append(cls)
            gt_boxes.append([cx, cy, w, h])

    if len(gt_boxes) == 0:
        print(f"[INFO] No GT objects in {img_name}, skipping.")
        continue

    # ---- 4.2 Run YOLO prediction ----
    results = model(img_path, verbose=False)[0]
    img_h, img_w = results.orig_shape

    pred_boxes = []
    pred_classes = []

    for b in results.boxes:
        xyxy = b.xyxy[0].cpu().numpy()
        cls = int(b.cls.cpu().numpy())
        pred_boxes.append(xyxy)
        pred_classes.append(cls)

    # ---- 4.3 Decide category (success / partial / missed) ----
    if len(pred_boxes) == 0:
        category = "missed"
    else:
        best_category = 0  # 0=missed, 1=partial, 2=success

        for gt_box, gt_cls in zip(gt_boxes, gt_classes):
            gt_xyxy = xywhn_to_xyxy(gt_box, img_w, img_h)
            best_iou_for_gt = 0.0

            for pb, pc in zip(pred_boxes, pred_classes):
                if pc != gt_cls:
                    continue
                current_iou = iou(gt_xyxy, pb)
                best_iou_for_gt = max(best_iou_for_gt, current_iou)

            if best_iou_for_gt >= IOU_SUCCESS:
                best_category = max(best_category, 2)
            elif IOU_PARTIAL <= best_iou_for_gt < IOU_SUCCESS:
                best_category = max(best_category, 1)

        category = (
            "success" if best_category == 2 else
            "partial" if best_category == 1 else
            "missed"
        )

    # ---- 4.4 Copy image into the corresponding folder ----
    if category == "success":
        dst_dir = SUCCESS_DIR
    elif category == "partial":
        dst_dir = PARTIAL_DIR
    else:
        dst_dir = MISSED_DIR

    shutil.copy2(img_path, os.path.join(dst_dir, img_name))
    print(f"{img_name}  -->  {category}")


[WARN] No best.pt matched all keywords; using first found:
        C:\Users\User\Documents\Rabab\Rubi's Project-20251021T100222Z-1-001\Rubi_s Project\yolov8l\finetuning-1\weights\best.pt


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User\\Documents\\Rabab\\Rubis Project-20251021T100222Z-1-001\\Rubi_s Project\\yolov8l\\finetuning-1\\weights\\best.pt'